In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/kir/KIR_development/bin/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/kir/KIR_development/bin/samples.ipynb")

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated since IPython 4.0. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)
/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


### Acquire samples to run

In [2]:
print len(normal_samples), len(normal_barcodes)

out_dirs = ['/nrnb/users/ramarty/TCGA/exomes/{0}'.format(x) for x in normal_barcodes]

out_dirs[:5]

normal_samples[:3], normal_barcodes[:3]

8333 8333


(['164511a9-2f56-49e0-b5cf-9c4be32f8fc7',
  '28a75d46-5b6d-4c40-8472-fccc77e3a2d9',
  '260e4af3-af8c-42e8-9914-059a08f9d8ae'],
 ['TCGA-HC-7737', 'TCGA-EJ-5515', 'TCGA-FP-8631'])

In [3]:
normal_barcodes[:5]

['TCGA-HC-7737',
 'TCGA-EJ-5515',
 'TCGA-FP-8631',
 'TCGA-85-8352',
 'TCGA-CV-6943']

### Expanded patient set

In [4]:
def get_TARGET(x):
    if 'TARGET' in x:
        return True
    else:
        return False
def get_origin(x):
    return int(x.split('-')[3][:2])

uuid_barcode_map = pd.read_csv('/cellar/users/andreabc/GDC_barcodes/uuid_barcode_map.txt', sep='\t')

# only exome
uuid_barcode_map = uuid_barcode_map[uuid_barcode_map.type == 'aligned_reads']
# remove target
uuid_barcode_map['TARGET'] = uuid_barcode_map.sample_barcode.apply(get_TARGET)
uuid_barcode_map = uuid_barcode_map[~uuid_barcode_map['TARGET']]

uuid_barcode_map['origin'] = uuid_barcode_map.sample_barcode.apply(get_origin)
uuid_barcode_map = uuid_barcode_map[uuid_barcode_map.origin.isin([10, 11])]

uuid_barcode_map = uuid_barcode_map.drop_duplicates('barcode')

In [5]:
len(uuid_barcode_map)

10505

In [6]:
uuid_barcode_map.head()

,file_id,file_name,barcode,sample_barcode,disease,type,data_format,TARGET,origin
180136,961f0d2a-6b74-47d6-9290-bbb5a3ff3822,C345.TCGA-BP-4967-11A-01D-1462-08.8_gdc_realn.bam,TCGA-BP-4967,TCGA-BP-4967-11A,TCGA-KIRC,aligned_reads,BAM,False,11
180138,2310be01-3a6b-4103-8bdf-418db1fdcb07,C494.TCGA-TM-A84O-10A-01D-A367-08.1_gdc_realn.bam,TCGA-TM-A84O,TCGA-TM-A84O-10A,TCGA-LGG,aligned_reads,BAM,False,10
180139,c9236e76-fd07-4946-b0b2-606464be1b34,C828.TCGA-BF-AAP2-10A-01D-A401-08.1_gdc_realn.bam,TCGA-BF-AAP2,TCGA-BF-AAP2-10A,TCGA-SKCM,aligned_reads,BAM,False,10
180142,ac9a4ab6-8752-42c6-b9fb-d338db61f830,C529.TCGA-EJ-7794-11A-01D-2114-08.1_gdc_realn.bam,TCGA-EJ-7794,TCGA-EJ-7794-11A,TCGA-PRAD,aligned_reads,BAM,False,11
180143,47b6b844-e690-4f61-b6f1-53a1b08d4df6,C495.TCGA-CV-A45Q-10A-01D-A24F-08.1_gdc_realn.bam,TCGA-CV-A45Q,TCGA-CV-A45Q-10A,TCGA-HNSC,aligned_reads,BAM,False,10


In [ ]:
# re-run the samples that failed

In [7]:
samples = list(uuid_barcode_map.file_id)
barcodes = list(uuid_barcode_map.barcode)
file_names = list(uuid_barcode_map.file_name)
out_dirs = ['/nrnb/users/ramarty/TCGA/exomes/{0}'.format(x) for x in barcodes]

In [8]:
len(samples), len(barcodes), len(out_dirs)

(10505, 10505, 10505)

### Create cluster scripts

Clean trash

In [27]:
def create_cluster_script_clean(samples, barcodes, out_dirs, file_names):

    new_script_file = '/cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/tcga/clean_tmp.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(samples)))
        #out_file.write("#$ -t 1-200\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=3G\n")
        out_file.write("#$ -tc 20\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set samples=({0})\n".format(" ".join(samples)))
        out_file.write("set barcodes=({0})\n".format(" ".join(barcodes)))
        out_file.write("set outs=({0})\n".format(" ".join(out_dirs)))
        out_file.write("set files=({0})\n".format(" ".join(file_names)))
        out_file.write("\n")

        out_file.write("set sample=$samples[$SGE_TASK_ID]\n")
        out_file.write("set barcode=$barcodes[$SGE_TASK_ID]\n")
        out_file.write("set out=$outs[$SGE_TASK_ID]\n")
        out_file.write("set file=$files[$SGE_TASK_ID]\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        
        # Clean up
        out_file.write("rm -r /tmp/ramarty/\n")
        out_file.write("rm -r /tmp/ramarty/*\n")
        out_file.write("date\n")

In [28]:
create_cluster_script_clean(samples, barcodes, out_dirs, file_names)

Download, pull sliced bam, pull unmapped, combine into single bam, finish

In [48]:
input_file = '/cellar/users/ramarty/Data/kir/ref/random_genes.fa'
names, sequences = [], []
fasta_sequences = SeqIO.parse(open(input_file),'fasta')
for fasta in fasta_sequences:
    name, sequence = fasta.description, fasta.seq.tostring()
    names.append(name)
    sequences.append(sequence)

df = pd.DataFrame({'Gene': names,
                  'Sequences': sequences})

fasta.description

regions = [x.split(', ')[1] for x in df.Gene]

regions_str = ' '.join(regions)

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/Bio/Seq.py:354: BiopythonDeprecationWarning: This method is obsolete; please use str(my_seq) instead of my_seq.tostring().
  BiopythonDeprecationWarning)


In [58]:
samtools = '/cellar/users/hcarter/programs/samtools-1.2/samtools'
def create_cluster_script_sliced(samples, barcodes, out_dirs, file_names):

    new_script_file = '/cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/tcga/exon_sliced.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(samples)))
        #out_file.write("#$ -t 9-9\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=8G\n")
        out_file.write("#$ -tc 30\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set samples=({0})\n".format(" ".join(samples)))
        out_file.write("set barcodes=({0})\n".format(" ".join(barcodes)))
        out_file.write("set outs=({0})\n".format(" ".join(out_dirs)))
        out_file.write("set files=({0})\n".format(" ".join(file_names)))
        out_file.write("\n")

        out_file.write("set sample=$samples[$SGE_TASK_ID]\n")
        out_file.write("set barcode=$barcodes[$SGE_TASK_ID]\n")
        out_file.write("set out=$outs[$SGE_TASK_ID]\n")
        out_file.write("set file=$files[$SGE_TASK_ID]\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        
        # Make directory
        out_file.write("mkdir $out\n")
        out_file.write("mkdir $out/features\n")
        out_file.write("mkdir /tmp/ramarty\n")
        out_file.write("mkdir /tmp/ramarty/$barcode\n")
        out_file.write("\n")
        
        # Download and process
        out_file.write("bash /cellar/users/ramarty/tokens/GDC.exome.sh $sample /tmp/ramarty/$barcode\n")
        out_file.write("echo Bam downloaded.\n")
        out_file.write("\n")
        
        # Pull KIR region bam
        out_file.write("{0} view -b /tmp/ramarty/$barcode/$sample/$file chr19:54025634-55084318 > ".format(samtools) + \
                       "/tmp/ramarty/$barcode/KIR.bam\n")
        out_file.write("echo Slice KIR region.\n")
        out_file.write("\n")
        
        # Pull random region bam
        out_file.write("{0} view -b /tmp/ramarty/$barcode/$sample/$file {1} > ".format(samtools, regions_str) + \
                       "/tmp/ramarty/$barcode/random_genes.bam\n")
        out_file.write("echo Slice random regions.\n")
        out_file.write("\n")
        
        # Pull unmapped bam
        out_file.write("{0} view -b -f 4 /tmp/ramarty/$barcode/$sample/$file > ".format(samtools) + \
                       "/tmp/ramarty/$barcode/unmapped.bam\n")
        out_file.write("echo Slice unmapped bam.\n")
        out_file.write("\n")
        
        # Combine into single bam
        out_file.write("{0} merge /tmp/ramarty/$barcode/KIR_and_unmapped.bam ".format(samtools) + \
                       "/tmp/ramarty/$barcode/KIR.bam /tmp/ramarty/$barcode/unmapped.bam " + \
                       "/tmp/ramarty/$barcode/random_genes.bam\n")
        out_file.write("echo Combined into single bam.\n")
        out_file.write("\n")
        
        # Strip fastq 
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/convert_to_fastq2.py " + \
                       "/tmp/ramarty/$barcode/KIR_and_unmapped.bam /tmp/ramarty/$barcode/KIR_sorted " + \
                       "/tmp/ramarty/$barcode/KIR_dedup /tmp/ramarty/$barcode/KIR_and_unmapped.fastq cellar\n")
        out_file.write("echo Fastq stripped.\n")
        out_file.write("\n")
        
        # Map to reference
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/map_to_reference.py " + \
                       "/tmp/ramarty/$barcode/KIR_and_unmapped.fastq $out/KIR_and_unmapped.aligned.bam " + \
                       "/cellar/users/ramarty/Data/kir/ref/all_alleles_and_random cellar\n")
        out_file.write("echo Mapped to KIR.\n")
        out_file.write("\n")
        
        # Clean up
        out_file.write("rm -r /tmp/ramarty/$barcode\n")
        out_file.write("date\n")

In [59]:
create_cluster_script_sliced(samples, barcodes, out_dirs, file_names)

In [15]:
barcodes[:10]

['TCGA-BP-4967',
 'TCGA-TM-A84O',
 'TCGA-BF-AAP2',
 'TCGA-EJ-7794',
 'TCGA-CV-A45Q',
 'TCGA-DH-5140',
 'TCGA-22-5479',
 'TCGA-DU-7306',
 'TCGA-HC-7078',
 'TCGA-EJ-8474']

Pull out components

In [16]:
def create_cluster_script_gather(barcodes, out_dirs):

    new_script_file = '/cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/tcga/gather_components.sh'
    
    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(barcodes)))
        #out_file.write("#$ -t 1-2\n".format(len(barcodes)))
        out_file.write("#$ -l h_vmem=2G\n")
        out_file.write("#$ -tc 10\n")
        out_file.write("#$ -pe smp 8\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set barcodes=({0})\n".format(" ".join(barcodes)))
        out_file.write("set outs=({0})\n".format(" ".join(out_dirs)))
        out_file.write("\n")

        out_file.write("set barcode=$barcodes[$SGE_TASK_ID]\n")
        out_file.write("set out=$outs[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("\n")
        
        # Make directories
        out_file.write("mkdir $out/features\n")
        out_file.write("\n")
        
        # Strip reads from KIR bam
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/convert_to_fastq2.py " + \
                       "$out/KIR_and_unmapped.aligned.bam $out/KIR_sorted $out/KIR_dedup " + \
                       "$out/KIR_and_unmapped.aligned.fastq cellar\n")
        out_file.write("echo Stripped reads.\n")
        out_file.write("\n")
        
        # Collect components        
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/component_collection.py " + \
                       "$out/KIR_and_unmapped.aligned.fastq $out/features " + \
                       "/cellar/users/ramarty/Data/kir/kmers/kmer_groups/kir_four_random.txt " + \
                       "kir_four_random\n")
        out_file.write("echo Components gathered - KIR.\n")
        out_file.write("\n")
        
        # Clean up
        out_file.write("rm $out/KIR_sorted\n")
        out_file.write("rm $out/KIR_dedup\n")
        out_file.write("rm $out/*fastq\n")
        out_file.write("date\n")

In [17]:
create_cluster_script_gather(barcodes, out_dirs)

# OLD
Download, convert to fastq, collect components, run PING, run HLA-HD <br>
Put the different results in different locations?? Might want to start fresh with barcodes

In [18]:
def create_cluster_script_download(samples, barcodes, out_dirs, file_names):

    new_script_file = '/cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/tcga/exon_download.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        #out_file.write("#$ -t 1-{0}\n".format(len(samples)))
        out_file.write("#$ -t 5-6\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=3G\n")
        out_file.write("#$ -tc 20\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set samples=({0})\n".format(" ".join(samples)))
        out_file.write("set barcodes=({0})\n".format(" ".join(barcodes)))
        out_file.write("set outs=({0})\n".format(" ".join(out_dirs)))
        out_file.write("set files=({0})\n".format(" ".join(file_names)))
        out_file.write("\n")

        out_file.write("set sample=$samples[$SGE_TASK_ID]\n")
        out_file.write("set barcode=$barcodes[$SGE_TASK_ID]\n")
        out_file.write("set out=$outs[$SGE_TASK_ID]\n")
        out_file.write("set file=$files[$SGE_TASK_ID]\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        
        # Make directory
        out_file.write("mkdir $out\n")
        out_file.write("mkdir $out/features\n")
        out_file.write("mkdir /tmp/ramarty\n")
        out_file.write("mkdir /tmp/ramarty/$barcode\n")
        
        # Download and process
        out_file.write("bash /cellar/users/ramarty/tokens/GDC.exome.sh $sample /tmp/ramarty/$barcode\n")
                      # "$sample $out/full_exome.bam\n")
        out_file.write("echo Bam downloaded.\n")
        
        # Strip fastq 
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/convert_to_fastq2.py " + \
                       "/tmp/ramarty/$barcode/$sample/$file /tmp/ramarty/$barcode/$sample/full_exome_sorted " + \
                       "/tmp/ramarty/$barcode/full_exome_1.fastq /tmp/ramarty/$barcode/full_exome_2.fastq cellar\n")
        out_file.write("echo Fastq stripped.\n")
        
        # Combine pairs
        out_file.write("cat /tmp/ramarty/$barcode/full_exome_1.fastq /tmp/ramarty/$barcode/full_exome_2.fastq " + \
                       "> /tmp/ramarty/$barcode/full_exome.fastq\n")
        out_file.write("echo Fastq combined.\n")
        
        # Map to reference
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/map_to_reference.py " + \
                       "/tmp/ramarty/$barcode/full_exome.fastq $out/full_exome_kir.bam " + \
                       "/cellar/users/ramarty/Data/kir/ref/all_alleles_and_random cellar\n")
        out_file.write("echo Mapped to KIR.\n")
        
        # Clean up
        out_file.write("rm -r /tmp/ramarty/$barcode\n")
        out_file.write("date\n")

In [19]:
create_cluster_script_download(samples, barcodes, out_dirs, file_names)

Data pull

In [ ]:
        # Run PING
        #out_file.write("cd /nrnb/users/ramarty/programs/PING\n")
        #out_file.write("Rscript --vanilla /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/runPING_extractor.R $out/ full_exome_1.fastq full_exome_2.fastq $out/PING_sequences/ 4\n")
        #out_file.write("Rscript --vanilla /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/runPING_gc_caller.R $out/PING_sequences/  $out/PING/ 40000\n")
        #out_file.write("echo PING completed.\n")
        #out_file.write("date\n")

Restart those that failed

In [79]:
job_id = '290648'

In [80]:
all_error_files = os.listdir('/cellar/users/ramarty/Data/kir/sge-system_files/')
relevant_error_files = [x for x in all_error_files if 'exon_download.sh.e{0}'.format(job_id) in x]

In [81]:
len(relevant_error_files)

205

In [82]:
jobs_to_restart = []
for f in relevant_error_files:
    lines = open('/cellar/users/ramarty/Data/kir/sge-system_files/' + f).readlines()
    try:
        if 'transfer closed' in lines[-1]:
            # minus one to reflect the position in the original list
            jobs_to_restart.append(int(f.split('.')[-1]) - 1)
    except:
        print f

exon_download.sh.e290648.204
exon_download.sh.e290648.203
exon_download.sh.e290648.202
exon_download.sh.e290648.205


In [83]:
jobs_to_restart = sort(jobs_to_restart)

In [84]:
len(jobs_to_restart)

19

In [69]:
def create_cluster_script_download_restart(samples, barcodes, out_dirs):

    new_script_file = '/cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/tcga/exon_download.rerun.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        #out_file.write("#$ -t 1-{0}\n".format(len(samples)))
        out_file.write("#$ -t 1-{0}\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=1G\n")
        out_file.write("#$ -tc 100\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set samples=({0})\n".format(" ".join(samples)))
        out_file.write("set barcodes=({0})\n".format(" ".join(barcodes)))
        out_file.write("set outs=({0})\n".format(" ".join(out_dirs)))
        out_file.write("\n")

        out_file.write("set sample=$samples[$SGE_TASK_ID]\n")
        out_file.write("set barcode=$barcodes[$SGE_TASK_ID]\n")
        out_file.write("set out=$outs[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        
        # Make directory
        out_file.write("mkdir $out\n")
        
        # Download and process
        out_file.write("bash /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/GDC.exome.sh $sample $out/full_exome.bam\n")
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/convert_to_fastq.py $out/full_exome.bam $out/full_exome.fastq cellar\n")
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/convert_to_fastq2.py $out/full_exome.bam $out/full_exome_1.fastq $out/full_exome_1.fastq cellar\n")
        
        out_file.write("date\n")

In [70]:
create_cluster_script_download_restart([normal_samples[i] for i in jobs_to_restart], [normal_barcodes[i] for i in jobs_to_restart], [out_dirs[i] for i in jobs_to_restart])